In [1]:
import pandas as pd

In [2]:
gatrain = pd.read_csv('data/gender_age_train.csv')
pbrand = pd.read_csv('data/phone_brand_device_model.csv')

In [3]:
gatest = pd.read_csv('data/gender_age_test.csv')

In [4]:
len(pbrand)

187245

In [5]:
len(gatrain)

74645

In [6]:
len(gatrain) + len(gatest)

186716

In [7]:
gatrain.columns

Index(['device_id', 'gender', 'age', 'group'], dtype='object')

In [8]:
pbrand.columns

Index(['device_id', 'phone_brand', 'device_model'], dtype='object')

In [9]:
gaptrain = gatrain.merge(pbrand, on='device_id')

In [10]:
gaptrain.columns

Index(['device_id', 'gender', 'age', 'group', 'phone_brand', 'device_model'], dtype='object')

In [11]:
gaptrain.head()

,device_id,gender,age,group,phone_brand,device_model
0,-8076087639492063270,M,35,M32-38,小米,MI 2
1,-2897161552818060146,M,35,M32-38,小米,MI 2
2,-8260683887967679142,M,35,M32-38,小米,MI 2
3,-4938849341048082022,M,30,M29-31,小米,红米note
4,245133531816851882,M,30,M29-31,小米,MI 3


In [12]:
gatest.head()

,device_id
0,1002079943728939269
1,-1547860181818787117
2,7374582448058474277
3,-6220210354783429585
4,-5893464122623104785


In [13]:
translations = pd.read_csv('data/phone_brand_chinese_to_english.csv', sep=',')

In [15]:
translations.head()

,ch,eng
0,三星,samsung
1,天语,Ktouch
2,海信,hisense
3,联想,lenovo
4,欧比,obi


In [19]:
translations.lookup(translations.index, translations['ch'])

KeyError: 'One or more column labels was not found'

In [30]:
pbrand.device_id = pbrand.device_id.astype('category')
pbrand.device_id.describe()

count                  187245
unique                 186716
top       1369352973364052752
freq                        2
Name: device_id, dtype: int64

In [33]:
pbrand.device_id.values

[-8890648629457979026, 1277779817574759137, 5137427614288105724, 3669464369358936369, -5019277647504317457, ..., 7979541072208733273, -187404680852357705, -2718274279595622821, 3098391762071677791, 7540390258631417003]
Length: 187245
Categories (186716, int64): [-9223321966609553846, -9223067244542181226, -9223042152723782980, -9222956879900151005, ..., 9222779211060772275, 9222784289318287993, 9222849349208140841, 9223069070668353002]

In [ ]:
gatrain.device_id = gatrain.device_id.astype('category')

In [37]:
criterion = pbrand['device_id'].map(lambda x: x in gatrain['device_id'].values)

len(pbrand[criterion])

74839

0